In [1]:
from data_classes import LAQNData, HealthData
from os import path
import pandas as pd

### Load the hourly NO$_2$ data:

In [2]:
data_folder = path.join(path.abspath(""), "LAQN_data")
species = "NO2"
url = "http://api.erg.kcl.ac.uk/AirQuality/Information/MonitoringSites/GroupName=London/Json"
start_date = "2000-01-01"
end_date = "2021-01-01"

In [3]:
NO2_hourly = LAQNData(url, data_folder, species, start_date, end_date)

In [4]:
hourly_df = NO2_hourly.read_csv(index_col="date")
print(hourly_df.shape)

Reading NO2_hourly_2000-01-01_2021-01-01.csv...
(184105, 192)


In [5]:
hourly_df.head()

,TD0,BG3,BG1,BG2,BN2,BN3,BN1,BX5,BX2,BQ7,...,WMD,WM0,MY1,WM6,WMZ,WMB,NB1,WM8,WM9,VS1
date,,,,,,,,,,,,,,,,,,,,,
2000-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,34.4,NaN,NaN,NaN,...,NaN,NaN,78.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-01 01:00:00,20.3,NaN,NaN,NaN,NaN,NaN,46.9,NaN,34.3,NaN,...,NaN,NaN,84.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-01 02:00:00,21.2,NaN,NaN,NaN,NaN,NaN,48.5,33.4,27.3,NaN,...,NaN,NaN,115.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-01 03:00:00,21.2,NaN,NaN,NaN,NaN,NaN,34.0,20.3,18.6,NaN,...,NaN,NaN,97.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-01 04:00:00,20.5,NaN,NaN,NaN,NaN,NaN,25.2,18.7,19.7,NaN,...,NaN,NaN,74.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Load the weekly mortality per capita data:

In [9]:
data_folder = path.join(path.abspath(""), "mortality_data")
filename = "weekly_mortality_London.csv"

In [11]:
mortality_weekly = HealthData(data_folder, filename=filename)

In [12]:
mortality_df = mortality_weekly.read_csv(index_col="date")
print(mortality_df.shape)

Reading weekly_mortality_London.csv...
(581, 1)


In [13]:
mortality_df.head()

,weekly_deaths
date,
2010-01-08,1226
2010-01-15,1262
2010-01-22,1186
2010-01-29,1093
2010-02-05,1035
